In [ ]:
import os
from pathlib import Path
from typing import Dict, Optional, List, Union, Tuple
from dataclasses import dataclass
import math
import numpy as np
import pandas as pd
from datasets import Dataset
from tqdm import tqdm
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import  DataLoader

from transformers.modeling_outputs import BaseModelOutputWithPastAndCrossAttentions
from transformers.pytorch_utils import apply_chunking_to_forward
from transformers.activations import ACT2FN
import pytorch_lightning as pl
import torchmetrics as tm
# import bitsandbytes as bnb
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
NODE_OP_CODES = 120
NODE_FEATS = 140
CONFIG_FEATS = 24
NODE_CONFIG_FEATS = 18
DATA_DIR = "../input/predict-ai-model-runtime/npz_all/npz"


def generate_tile_df() -> pd.DataFrame:
    tile_df = pd.DataFrame({'paths': [elem for elem in (Path(DATA_DIR) / 'tile').rglob("*") if elem.is_file()]}).assign(
        split=lambda df: df.paths.apply(lambda x: x.parent.name),
        configuration=lambda df: df.paths.apply(lambda x: x.parent.parent.name),
        extra=lambda df: df.paths.apply(lambda x: x.parent.parent.parent.name),
        model_name=lambda df: df.paths.apply(lambda x: x.stem),
        collection=lambda df: df.extra + ':' + df.configuration ,
        ID=lambda df: df.collection + ':' + df.model_name ,
        paths = lambda df: df.paths.apply(lambda x: str(x))
    )
    return tile_df
tile_df = generate_tile_df()
tile_df.head()
"""
paths	split	configuration	extra	model_name	collection	ID
0	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	resnet_v1_50_official_batch_128_bf16_2bea628b7...	tile:xla	tile:xla:resnet_v1_50_official_batch_128_bf16_...
1	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	inception_v3_batch_128_train_40fa8f86f121f00a	tile:xla	tile:xla:inception_v3_batch_128_train_40fa8f86...
2	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	inception_v3_batch_128_train_-23e94c034a65a177	tile:xla	tile:xla:inception_v3_batch_128_train_-23e94c0...
3	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	inception_v3_batch_128_train_171f4371caf28639	tile:xla	tile:xla:inception_v3_batch_128_train_171f4371...
4	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	mlperf_bert_batch_24_2x2_-25e30862c042a2b8	tile:xla	tile:xla:mlperf_bert_batch_24_2x2_-25e30862c04...
"""
#Dataset
#Create an Adjacency matrix for masking the attention
#Creates a virtual first node equivalent to the [CLS] token which contains the global config for tile cases, while layout node configuration goes to the corresponding node position
def edges_adjacency(edges: torch.Tensor, add_diagonal=True) -> torch.Tensor:
    """
    Generate an adjacency matrix from the edges
    Args:
        edges: Tensor of shape (num_edges, 2) with the edges
        add_diagonal: Boolean indicating if the diagonal should be added to the adjacency matrix
    Returns:
        adjacency_matrix: Tensor of shape (num_nodes, num_nodes) with the adjacency matrix
    """
    adjacency_matrix = torch.zeros((edges.max() + 1, edges.max() + 1))
    adjacency_matrix[edges[:, 0], edges[:, 1]] = 1
    if add_diagonal:
        diag_idx = torch.arange(adjacency_matrix.shape[0])
        adjacency_matrix[diag_idx, diag_idx] = 1
    return adjacency_matrix

def tile_loader(path):
    tile_dict =  dict(np.load(path))
    tile_dict = {k: torch.from_numpy(v) for k, v in tile_dict.items()}
    tile_dict['edges_adjecency'] = edges_adjacency(tile_dict['edge_index'])
    return tile_dict

def node_cls_token(elem_dict, shift_node_config_ids:bool=True):
    """
    Add a cls token to the node opcode, features, edges adjacency matrix, shift node_config_ids by 1 to account for the cls token
    Args:
        elem_dict: Dictionary with the elements of the tile
    Returns:
        elem_dict: Dictionary with the elements of the tile with the cls token
    """
    elem_dict['node_opcode'] = torch.cat([torch.tensor([0]), elem_dict['node_opcode']]) # Introduce [CLS] node
    elem_dict['node_feat'] = torch.cat([torch.zeros((1, elem_dict['node_feat'].shape[1])), elem_dict['node_feat']])
    elem_dict['edges_adjecency'] = F.pad(elem_dict['edges_adjecency'], (1,0,1,0), value=1)
    if 'node_config_ids' in elem_dict and shift_node_config_ids:
        elem_dict['node_config_ids'] = elem_dict['node_config_ids'] + 1 # Shift Node Config IDs to take in to account [CLS] node
    return elem_dict


class TileDataset(torch.utils.data.Dataset):
    
    def __init__(self, df:pd.DataFrame ,add_cls_token:bool=True, num_configs:int=10,  max_configs:Optional[int]=None):
        self.df = df
        self.add_cls_token = add_cls_token
        self.num_configs = num_configs
        self.max_configs = max_configs  
        
    def __len__(self) -> int:
        return len(self.df)
    
    def select_configs(self, total_configs:int):
        if self.max_configs is not None:
            total_configs = min(total_configs, self.max_configs)
        if self.num_configs == -1:
            return np.arange(total_configs)
        if total_configs < self.num_configs:
            return np.random.choice(total_configs, self.num_configs, replace=True)
        return  np.random.choice(total_configs, self.num_configs, replace=False)
    
    def __getitem__(self, idx:int, selected_configs:List[int]=None):
        tile_dict = tile_loader(self.df.paths[idx])
        if selected_configs is None:
            selected_configs = self.select_configs(tile_dict['config_feat'].shape[0])
        tile_dict['node_config_feat'] = tile_dict.pop('config_feat')[selected_configs]
        tile_dict['node_config_feat'] = F.pad(tile_dict['node_config_feat'].unsqueeze(1), (0,NODE_CONFIG_FEATS))
        tile_dict['config_runtime'] = tile_dict['config_runtime'][selected_configs].float()
        tile_dict['config_runtime'] /= tile_dict['config_runtime_normalizers'][selected_configs].float()
        tile_dict['node_config_ids'] = torch.zeros((1,))
        tile_dict['selected_idxs'] = selected_configs
        if self.add_cls_token:
            tile_dict = node_cls_token(tile_dict, False)
        return tile_dict
"""
edges_adjacency関数: 入力: エッジのテンソルと対角線を追加するかどうかのブール値フラグ。 処理: エッジのテンソルから隣接行列を生成します。対角線を追加するオプションもあります。 出力: 隣接行列のテンソル。
tile_loader関数: 入力: パス（データセットのエレメントが保存されている場所）。 処理: NumPyファイルをロードし、その内容をPyTorchのテンソルに変換します。さらに、エッジの隣接行列も生成します。 出力: エレメントの辞書（テンソルに変換され、隣接行列も含まれている）。
node_cls_token関数: 入力: タイルのエレメントの辞書。 処理: [CLS]トークンをノードのオペコード、特徴、エッジの隣接行列に追加します。また、ノードの設定IDも1だけシフトします。 出力: [CLS]トークンが追加されたタイルのエレメントの辞書。
TileDatasetクラス: 役割: タイルのデータセットを管理し、データのロードと前処理を担当します。 initメソッド: インスタンスを初期化し、データフレーム、[CLS]トークンの追加、設定の数などのパラメータを設定します。 lenメソッド: データセットの長さ（エレメントの総数）を返します。 select_configsメソッド: 総設定から特定の設定を選択します。 getitemメソッド: インデックスに基づいてデータセットからエレメントを取得し、必要に応じて前処理を行います。 処理の流れ: グラフのエッジから隣接行列を生成します。 NumPyファイルからタイルデータをロードし、PyTorchのテンソルに変換します。 [CLS]トークンを追加して、ノードやエッジのデータを修正・更新します。 TileDatasetクラスを使用して、データのロードと前処理を効率的に行います。 このコードは、グラフのノードとエッジのデータを効率的に処理して、モデルのトレーニングや評価に使用するためのデータセットを準備するためのものです。
"""

tile_dataset = TileDataset(tile_df)
elem = tile_dataset[0]
for k,v in elem.items():
    print(k, v.shape)
node_feat torch.Size([81, 140])
node_opcode torch.Size([81])
edge_index torch.Size([86, 2])
config_runtime torch.Size([10])
config_runtime_normalizers torch.Size([3246])
edges_adjecency torch.Size([81, 81])
node_config_feat torch.Size([10, 1, 42])
node_config_ids torch.Size([1])
selected_idxs (10,)
elem['edges_adjecency']
"""
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 1., 1., 1.]])
"""
#Collator
def pad_edge_adjacency(edges_adjacency_list):
    max_len = max([elem.shape[0] for elem in edges_adjacency_list])
    return torch.stack([F.pad(elem, (0, max_len-elem.shape[0], 0, max_len-elem.shape[0]), value=0) for elem in edges_adjacency_list], dim=0)

@dataclass
class LayoutCollator:
    pad_to_multiple_of: int = 64
    targets:bool = True
    padding_idx:int = 120
    node_padding_idx:int = 0
    
    def __call__(self, batch):
        output = {}
        max_node_len = max([elem['node_opcode'].shape[0] for elem in batch])
        node_pad_amount = self.pad_to_multiple_of - max_node_len % max(self.pad_to_multiple_of, 1)
        output['node_opcode'] = F.pad(pad_sequence([elem['node_opcode'] for elem in batch], batch_first=True, padding_value=self.padding_idx),
                                      (0, node_pad_amount), value=self.padding_idx).long()
        output['node_feat'] = F.pad(pad_sequence([elem['node_feat'] for elem in batch], batch_first=True),
                                    (0,0,0, node_pad_amount), value=0)
        output['edges_adjecency'] = F.pad(pad_edge_adjacency([elem['edges_adjecency'] for elem in batch]),
                                          (0, node_pad_amount, 0, node_pad_amount), value=0)
        output['node_attn_mask'] = F.pad(pad_sequence([torch.ones(len(elem['node_opcode'])) for elem in batch], batch_first=True),
                                         (0, node_pad_amount), value=0)

        max_node_config_len = max([elem['node_config_ids'].shape[0] for elem in batch])
        node_config_pad_amount = self.pad_to_multiple_of - max_node_config_len % max(self.pad_to_multiple_of, 1)
        output['node_config_ids'] = F.pad(pad_sequence([elem['node_config_ids'] for elem in batch], batch_first=True),
                                         (0, node_config_pad_amount), value=0).long()
        padded_node_config_feat = pad_sequence([elem['node_config_feat'].permute(1,0,2) for elem in batch], batch_first=True, padding_value=-1)
        padded_node_config_feat = F.pad(padded_node_config_feat.permute(0,2,1,3),
                                           (0,0,0, node_config_pad_amount,0,0), value=-1)
        
        output['node_config_feat'] = torch.where(padded_node_config_feat!=-1, padded_node_config_feat, self.node_padding_idx)
                                      
        output['config_idxs'] = torch.stack([torch.from_numpy(elem['selected_idxs']) for elem in batch])
        
        if self.targets:
            output['config_runtime'] = pad_sequence([elem['config_runtime'].float() for elem in batch], batch_first=True)
        return output
"""
グラフベースのデータセットのバッチ処理を効率的に行うためのデータコレータクラスとその補助関数を定義しています。データコレータは、バッチ内のデータ要素を適切にパディングして、バッチ処理を効率化する役割があります。

pad_edge_adjacency関数: 入力: エッジの隣接行列のリスト。 処理: リスト内のすべての隣接行列を、最大の行列サイズに合わせてパディングします。 出力: パディングされた隣接行列のバッチ。
LayoutCollatorクラス: 役割: バッチ内のデータを適切にパディングして、一貫した形状のテンソルに変換します。これにより、バッチ処理が効率的に行えます。 pad_to_multiple_of属性: パディングされたデータの長さがこの値の倍数になるように設定します。 targets属性: ターゲット（例: ランタイム）も出力に含めるかどうかを制御します。 padding_idx属性: オペコードのパディングに使用するインデックス値。 node_padding_idx属性: ノードの特徴量のパディングに使用する値。 callメソッド: バッチのデータ処理とパディングを行います。 入力: バッチデータ。 処理: 各データエレメントのノード、エッジ、特徴量などをパディングして、一貫した形状にします。 出力: パディングされ、整形されたバッチデータ。 具体的な処理内容: ノードのオペコードのパディング: バッチ内のすべてのデータエレメントでノードのオペコードを最大の長さにパディングします。 ノードの特徴量のパディング: バッチ内のすべてのデータエレメントでノードの特徴量を最大の長さにパディングします。 エッジの隣接行列のパディング: pad_edge_adjacency関数を使用して、エッジの隣接行列をパディングします。 ノードの設定IDと特徴量のパディング: バッチ内のすべてのデータエレメントでノードの設定IDと特徴量を最大の長さにパディングします。 ランタイムのパディング: ターゲットとして使用するランタイムをパディングします（targets属性がTrueの場合）。 このクラスと関数は、バッチ処理中にデータの形状を一貫させ、モデルのトレーニングや評価を効率的に行うために使用されます。
"""

collate_fn = LayoutCollator(64)
batch = collate_fn([tile_dataset[0], tile_dataset[1]])
for k,v in batch.items():
    print(k,v.shape)
node_opcode torch.Size([2, 128])
node_feat torch.Size([2, 128, 140])
edges_adjecency torch.Size([2, 128, 128])
node_attn_mask torch.Size([2, 128])
node_config_ids torch.Size([2, 64])
node_config_feat torch.Size([2, 10, 64, 42])
config_idxs torch.Size([2, 10])
config_runtime torch.Size([2, 10])
Model - Config
@dataclass
class GraphConfig:
    num_hidden_layers: int = 8
    hidden_size: int = 256
    num_attention_heads: int = 16
    intermediate_size: int = 64
    chunk_size_feed_forward: int = 64
    attention_probs_dropout_prob: float = 0.0
    max_position_embeddings: int = 512
    hidden_dropout_prob: float = 0.0
    layer_norm_eps: float = 1e-12
    hidden_act: str = 'gelu'
    initializer_range: float = 0.02
    output_hidden_states: bool = False
    output_attentions: bool = False
    gradient_checkpointing: bool = False
    margin: float = 0.1
    number_permutations: int = 10
    
    def __post_init__(self):
        self.embedding_size = self.hidden_size
    
    def validate(self):
        if self.hidden_size % self.num_attention_heads != 0 and not hasattr(self, "embedding_size"):
            raise ValueError(
                f"The hidden size ({self.hidden_size}) is not a multiple of the number of attention "
                f"heads ({self.num_attention_heads})"
            )
            
    def save_config(self, path):
        config = asdict(self)
        with open(path, 'w') as f:
            json.dump(config, f)
            
    @classmethod
    def load_config(cls, path):
        with open(path, 'r') as f:
            config = json.load(f)
        return cls(**config)

"""
グラフネットワークまたはトランスフォーマーモデルの設定を管理するGraphConfigクラスを定義しています。このクラスは、モデルのハイパーパラメータと設定を格納、検証、保存、ロードする機能を提供します。

GraphConfig クラスの属性： num_hidden_layers: モデルの隠れ層の数。 hidden_size: 隠れ層のユニット数（次元数）。 num_attention_heads: アテンションヘッドの数。 intermediate_size: インターメディエイトレイヤー（フィードフォワードネットワーク部分）のサイズ。 chunk_size_feed_forward: フィードフォワードネットワークのチャンクサイズ。 attention_probs_dropout_prob: アテンション確率のドロップアウト率。 max_position_embeddings: 最大位置埋め込みのサイズ。 hidden_dropout_prob: 隠れ層のドロップアウト率。 layer_norm_eps: レイヤー正規化のepsilon（安定性のための小さい値）。 hidden_act: 隠れ層の活性化関数。 initializer_range: 重みの初期化の範囲。 output_hidden_states: 隠れ状態を出力するかどうか。 output_attentions: アテンションを出力するかどうか。 gradient_checkpointing: 勾配のチェックポイントを使用するかどうか（メモリ効率のため）。 margin: ロス計算で使用するマージン。 number_permutations: パーミュテーションの数。

GraphConfig クラスのメソッド： post_init メソッド: インスタンスの初期化が完了した後に、embedding_size を hidden_size と同じ値で設定します。 validate メソッド: モデルの設定が正しいことを確認します。特に、hidden_size が num_attention_heads の倍数であることを確認します。 save_config メソッド: モデルの設定をJSONファイルとして保存します。 入力: 設定を保存するパス。 処理: 設定ディクショナリをJSONとしてファイルに書き込みます。 load_config クラスメソッド: JSONファイルからモデルの設定をロードします。 入力: 設定ファイルのパス。 出力: ロードされた設定を持つ新しい GraphConfig インスタンス。

Loss
Uses Ranking loss to compare different configuration
Compares does configurations with different indexes, masks those cases where the permutation returns the same element
Compares multiple configurations in each run
"""
class MultiElementRankLoss(nn.Module):
    """
    Loss function that compares the output of the model with the output of the model with a permutation of the elements
    """
    
    def __init__(self, margin:float=0.0, number_permutations:int = 1) -> None:
        super().__init__()
        self.loss_fn = torch.nn.MarginRankingLoss(margin=margin, reduction = 'none')
        self.number_permutations = number_permutations
    
    def calculate_rank_loss(self,
                            outputs: torch.Tensor,
                            config_runtime: torch.Tensor,
                            config_idxs: torch.Tensor
                            ):
        """
        Generates a permutation of the predictions and targets and calculates the loss MarginRankingLoss against the permutation
        Args:
            outputs: Tensor of shape (bs, seq_len) with the outputs of the model
            config_runtime: Tensor of shape (bs, seq_len) with the runtime of the model
            config_mask: Tensor of shape (bs, seq_len) with 1 in the positions of the elements
            and 0 in the positions of the padding
        Returns:
            loss: Tensor of shape (bs, seq_len) with the loss for each element in the batch
        """
        bs, num_configs = outputs.shape
        permutation = torch.randperm(num_configs) 
        permuted_idxs = config_idxs[:, permutation]
        # We mask those cases where we compare the same configuration
        config_mask = torch.where(config_idxs != permuted_idxs, 1, 0)
        permuted_runtime = config_runtime[:, permutation]
        labels = 2*((config_runtime - permuted_runtime) > 0) -1
        permuted_output = outputs[:, permutation]
        loss = self.loss_fn(outputs.view(-1,1), permuted_output.view(-1,1), labels.view(-1,1))
        loss = loss.view(bs, num_configs) * config_mask
        return loss.mean()
                
    
    def forward(self,
                outputs: torch.Tensor,
                config_runtime: torch.Tensor,
                config_idxs: torch.Tensor
                ):
        loss = 0 
        for _ in range(self.number_permutations):
            loss += self.calculate_rank_loss(outputs, config_runtime, config_idxs)
        return loss/ self.number_permutations
"""
このコードは、複数のエレメントのランキング損失（MultiElementRankLoss）を計算するPyTorchのカスタム損失関数クラスを定義しています。この損失関数は、モデルの出力とエレメントの順序を変更したモデルの出力を比較し、ランキングが正しくない場合にペナルティを与えます。

クラスとメソッドの詳細：

MultiElementRankLoss クラス:
目的: モデルの出力とエレメントの順序を変更したモデルの出力を比較して損失を計算する。

init メソッド:
入力: margin と number_permutations。 処理: MarginRankingLoss を初期化し、パラメータを設定する。

calculate_rank_loss メソッド:
入力: outputs, config_runtime, config_idxs。 outputs: モデルの出力。 config_runtime: 各エレメントのランタイム。 config_idxs: 各エレメントのインデックス。 処理: エレメントの順序をランダムに並べ替えて、MarginRankingLoss を計算する。この並べ替えはランキングの精度を評価するために行われます。 出力: 各バッチエレメントの平均損失。

入力: outputs, config_runtime, config_idxs。 処理: 指定された数の順列で calculate_rank_loss を繰り返し、平均損失を計算する。 出力: 最終的な平均損失。 クラスの動作: インスタンス化: margin と number_permutations を指定してクラスをインスタンス化します。margin はマージンランキング損失のマージン、number_permutations は順列の数です。 損失計算: forward メソッドを呼び出して損失を計算します。これにはモデルの出力、設定のランタイム、設定のインデックスが必要です。 順列と損失: calculate_rank_loss メソッドは、モデルの出力とその順列を使ってマージンランキング損失を計算します。同じ設定で比較されるケースはマスクされ、損失の計算には含まれません。 平均損失: すべての順列に対する損失の平均が最終的な損失として返されます。 このカスタム損失関数は、モデルがグラフのエレメントを正しくランク付けする能力を評価と最適化するために使われます。

"""
#Metric
class TileTopK(tm.Metric):
    
    higher_is_better = True
    
    def __init__(self, k:int=5) -> None:
        super().__init__()
        self.add_state("runtimes", default=[], dist_reduce_fx=None)
        self.k = k
        
    def update(self, preds: torch.Tensor, target: torch.Tensor, config_attn_mask:torch.Tensor) -> None:
        """
        Update the metric state
        Args:
            preds: Tensor of shape (bs, seq_len) with the predicted runtimes orders
            target: Tensor of shape (bs, seq_len) with the target runtimes
            config_attn_mask: Tensor of shape (bs, seq_len) with 1 in the positions of the elements
        """
        best_runtimes = torch.where(config_attn_mask==1, target, torch.tensor(float('inf'))).min(1).values
        masked_preds = torch.where(config_attn_mask==1, preds, torch.tensor(float('inf')))
        pred_bottomk_indices = torch.topk(masked_preds, k=self.k, largest=False).indices
        bs = preds.shape[0]
        bottom_k_positions = torch.stack([torch.arange(bs).repeat_interleave(self.k).to(config_attn_mask.device), pred_bottomk_indices.view(-1)])
        predicted_runtimes = target[bottom_k_positions[0], bottom_k_positions[1]].view(bs,self.k)
        best_predicted_runtimes = predicted_runtimes.min(1).values
        self.runtimes.append(best_predicted_runtimes/ best_runtimes)
        
    def compute(self) -> torch.Tensor:
        return (2-torch.cat(self.runtimes)).mean()
"""
モデルが予測したランタイムとターゲットランタイムを比較して、ランタイムの予測の正確さを評価するためのカスタムメトリクスを定義しています。具体的には、Top-K メトリクスを使用して、モデルが予測したトップKの設定のランタイムと、実際の最適なランタイムとを比較します。

TileTopK クラスの説明：

属性とメソッド:
higher_is_better: このメトリクスが高いほど良い、という意味でTrueに設定されています。 init メソッド: 初期化メソッドで、Top-KのKの値を設定します。 update メソッド: 予測とターゲットのランタイム、および設定のアテンションマスクを取得して、メトリクスの状態を更新します。 compute メソッド: 現在のメトリクスの状態から最終的なメトリクスの値を計算します。

update メソッドの動作:
入力: preds: モデルによって予測されたランタイムの順序。 target: 実際のランタイムの目標値。 config_attn_mask: エレメントの位置に1、パディングの位置に0が入ったアテンションマスク。

処理: アテンションマスクを使用して、有効な設定の位置のランタイムの最小値（最適なランタイム）を計算します。 予測されたランタイムのうち、Top-Kの設定を選択します。 選択されたTop-K設定の予測ランタイムと実際の最適なランタイムを比較します。 この比較の結果を状態に保存します。

compute メソッドの動作:
処理: update メソッドで保存されたすべての比較結果から、最終的なメトリクスの値を計算します。 出力: 計算されたメトリクスの値を返します。 使い方のシナリオ： モデルの訓練または評価中に、このTileTopKメトリクスを使用して、モデルのランタイム予測の性能を評価します。 それぞれのバッチで、updateメソッドを呼び出して、予測とターゲットのランタイム、および設定のアテンションマスクを渡します。これにより、メトリクスの状態が更新されます。 評価が完了したら、computeメソッドを呼び出して、最終的なメトリクスの値を取得します。これがモデルのランタイム予測の性能を示す値になります。 このメトリクスは、モデルがどれだけ正確にランタイムを予測できるか、特にトップKの設定に焦点を当てて評価するのに役立ちます。
"""
#Model
#Modified version of 🤗 Bert implementation to take in to account Graph Attention
#
#Removed the parts corresponding to Cross-attention
#Made layer_head_mask the same for all layers, heads
#The Head mask corresponds to the edge adjacency
# Modified from https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py

class BertEncoder(nn.Module):
    def __init__(self, config:GraphConfig):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])
        self.gradient_checkpointing = False

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = False,
        output_hidden_states: Optional[bool] = False,
        return_dict: Optional[bool] = True,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None

        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask #DONE: Same Head Mask for all layers

            if self.gradient_checkpointing and self.training:

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs,  output_attentions)

                    return custom_forward

                layer_outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer_module),
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    output_attentions,
                )

            hidden_states = layer_outputs[0]
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [
                    hidden_states,
                    all_hidden_states,
                    all_self_attentions,
                ]
                if v is not None
            )
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=None,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=None,
        )
        
        
class BertLayer(nn.Module):
    def __init__(self, config:GraphConfig):
        super().__init__()
        self.chunk_size_feed_forward = config.chunk_size_feed_forward
        self.seq_len_dim = 1
        self.attention = BertAttention(config)
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attention_outputs = self.attention(
            hidden_states,
            attention_mask,
            head_mask,
            output_attentions=output_attentions,
        )
        attention_output = self_attention_outputs[0]
        outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights
        layer_output = apply_chunking_to_forward(
            self.feed_forward_chunk, self.chunk_size_feed_forward, self.seq_len_dim, attention_output
        )
        outputs = (layer_output,) + outputs


        return outputs

    def feed_forward_chunk(self, attention_output):
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output
    
class BertIntermediate(nn.Module):
    def __init__(self, config:GraphConfig):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        if isinstance(config.hidden_act, str):
            self.intermediate_act_fn = ACT2FN[config.hidden_act]
        else:
            self.intermediate_act_fn = config.hidden_act

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states
    
class BertOutput(nn.Module):
    def __init__(self, config:GraphConfig):
        super().__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states: torch.Tensor, input_tensor: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states
    
class BertAttention(nn.Module):
    def __init__(self, config:GraphConfig, position_embedding_type=None):
        super().__init__()
        self.self = BertSelfAttention(config, position_embedding_type=position_embedding_type)
        self.output = BertSelfOutput(config)
        self.pruned_heads = set()


    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        self_outputs = self.self(
            hidden_states,
            attention_mask,
            head_mask,
            output_attentions,
        )
        attention_output = self.output(self_outputs[0], hidden_states)
        outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them
        return outputs
    
    
class BertSelfAttention(nn.Module):
    def __init__(self, config:GraphConfig, position_embedding_type=None):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention "
                f"heads ({config.num_attention_heads})"
            )

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        self.position_embedding_type = position_embedding_type or getattr(
            config, "position_embedding_type", "absolute"
        )
        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            self.max_position_embeddings = config.max_position_embeddings
            self.distance_embedding = nn.Embedding(2 * config.max_position_embeddings - 1, self.attention_head_size)


    def transpose_for_scores(self, x: torch.Tensor) -> torch.Tensor:
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        
        mixed_query_layer = self.query(hidden_states)
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))
        query_layer = self.transpose_for_scores(mixed_query_layer)


        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            query_length, key_length = query_layer.shape[2], key_layer.shape[2]
            position_ids_l = torch.arange(query_length, dtype=torch.long, device=hidden_states.device).view(-1, 1)
            position_ids_r = torch.arange(key_length, dtype=torch.long, device=hidden_states.device).view(1, -1)
            distance = position_ids_l - position_ids_r

            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask #DONE: Same Head Mask for all Heads

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        return outputs


class BertSelfOutput(nn.Module):
    def __init__(self, config:GraphConfig):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states: torch.Tensor, input_tensor: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states
    
    
class NodeEncoder(nn.Module):
    
    def __init__(self, config:GraphConfig):
        super().__init__()
        self.node_opcode_embeddings = nn.Embedding(NODE_OP_CODES+1 , config.embedding_size, padding_idx=NODE_OP_CODES)
        self.linear = nn.Linear(NODE_FEATS, config.embedding_size, bias=False)
        self.layer_norm = nn.LayerNorm(config.embedding_size, eps=config.layer_norm_eps)
        
        
    def forward(self,
                node_opcode: torch.Tensor,
                node_feat: torch.Tensor
                ) -> torch.Tensor:
        opcode_embeddings = self.node_opcode_embeddings(node_opcode) 
        node_feats =  self.linear(node_feat)
        features = opcode_embeddings + node_feats
        features = self.layer_norm(features)
        return features
    
    
class BertNodeEncoder(nn.Module):
    
    def __init__(self, config:GraphConfig) -> None:
        super().__init__()
        self.config = config
        self.node_embeddings = NodeEncoder(config)
        self.node_encoder = BertEncoder(config)
        
    def forward(self,
                node_opcode: torch.Tensor,
                node_feat: torch.Tensor,
                edges_adjecency: torch.Tensor,
                node_attn_mask: torch.Tensor
                ):
        node_embeddings = self.node_embeddings(node_opcode, node_feat)
        node_attn_mask = node_attn_mask.unsqueeze(1).unsqueeze(-1)
        node_encoder_outputs = self.node_encoder(node_embeddings,
                                                 attention_mask=node_attn_mask,
                                                 head_mask=edges_adjecency.unsqueeze(0).repeat(self.config.num_hidden_layers, 1, 1, 1).unsqueeze(2),
                                                 output_attentions=True)
        return node_encoder_outputs
    
def transform_node_positional_embeddings(embeddings_output:torch.Tensor,
                                         node_config_ids:torch.Tensor,
                                         num_nodes:int
                                         ) -> torch.Tensor:
    bs, num_configs, _, dim = embeddings_output.shape
    idxs = node_config_ids.unsqueeze(1).repeat(1,num_configs,1)
    zeros = torch.zeros(bs, num_configs, num_nodes, dim, device=embeddings_output.device, dtype=embeddings_output.dtype)
    idxs = idxs.unsqueeze(-1).repeat(1,1,1,dim)
    zeros.scatter_reduce_(2, idxs, embeddings_output, reduce='sum')
    return zeros

class NodeFeatEmbeddings(nn.Module):
    def __init__(self, config:GraphConfig):
        super().__init__()
        self.config = config
        self.node_feat_embeddings = nn.Linear(NODE_CONFIG_FEATS + CONFIG_FEATS, config.embedding_size, bias=False)
        self.layer_norm = nn.LayerNorm(config.embedding_size, eps=config.layer_norm_eps)
        
    def forward(self, node_config_feat: torch.Tensor, node_config_ids: torch.Tensor, num_nodes:int) -> torch.Tensor:
        node_config_feat_embeddings = self.node_feat_embeddings(node_config_feat)
        node_config_feat_embeddings = self.layer_norm(node_config_feat_embeddings)
        node_config_feat_embeddings = transform_node_positional_embeddings(node_config_feat_embeddings, node_config_ids, num_nodes)
        return node_config_feat_embeddings
        
    
class BertGraphEncoder(nn.Module):
    def __init__(self, config:GraphConfig) -> None:
        super().__init__()
        self.config = config
        self.node_embeddings = NodeEncoder(config)
        self.node_encoder = BertEncoder(config)
        self.node_feat_embeddings = NodeFeatEmbeddings(config)
        
    def forward(self,
                node_opcode: torch.Tensor, # (bs, num_nodes)
                node_feat: torch.Tensor, # (bs, num_nodes, num_node_feats)
                edges_adjecency: torch.Tensor, # (bs, num_nodes, num_nodes)
                node_attn_mask: torch.Tensor, # (bs, num_nodes)
                node_config_feat: torch.Tensor, # (bs, num_configs, num_config_nodes, num_node_feats)
                node_config_ids: torch.Tensor, # (bs, num_configs, num_config_nodes)
                ):
        bs, num_nodes = node_opcode.shape
        num_configs = node_config_feat.shape[1]
        node_embeddings = self.node_embeddings(node_opcode, node_feat)
        node_config_feat_embeddings = self.node_feat_embeddings(node_config_feat, node_config_ids, num_nodes)
        
        node_embeddings = node_embeddings.unsqueeze(1).repeat(1, num_configs, 1, 1)
        node_embeddings += node_config_feat_embeddings
        node_attn_mask = node_attn_mask.unsqueeze(1).repeat(1, num_configs, 1)
        node_embeddings = node_embeddings.reshape(bs *num_configs, num_nodes, -1)
        node_attn_mask = node_attn_mask.reshape(bs *num_configs, num_nodes)
        node_attn_mask = node_attn_mask.unsqueeze(1).unsqueeze(-1)
        edges_adjecency = edges_adjecency.unsqueeze(1).repeat(1, num_configs, 1, 1).reshape(bs *num_configs, num_nodes, num_nodes)
        edges_adjecency = edges_adjecency.unsqueeze(1)
        

        node_encoder_outputs = self.node_encoder(node_embeddings,
                                                 attention_mask=node_attn_mask,
                                                 head_mask=edges_adjecency,
                                                 output_attentions=True)
        
        return node_encoder_outputs.last_hidden_state.reshape(bs, num_configs, num_nodes, -1)
    
    
class GraphEncoder(nn.Module):
    
    config_class = GraphConfig
    
    def __init__(self, config:GraphConfig):
        super().__init__()
        self.config = config
        self.node_encoder = BertGraphEncoder(config)
        self.head = nn.Linear(config.hidden_size, 1)
        self.loss_fn = MultiElementRankLoss(margin=config.margin, number_permutations=config.number_permutations)
        
        
    def forward(self,
                node_opcode: torch.Tensor, # (bs, num_nodes)
                node_feat: torch.Tensor, # (bs, num_nodes, num_node_feats)
                edges_adjecency: torch.Tensor, # (bs, num_nodes, num_nodes)
                node_attn_mask: torch.Tensor, # (bs, num_nodes)
                node_config_feat: torch.Tensor, # (bs, num_configs, num_config_nodes, num_node_feats)
                node_config_ids: torch.Tensor, # (bs, num_configs, num_config_nodes)
                config_idxs: Optional[torch.Tensor] = None, # (bs, num_configs)
                config_runtime: Optional[torch.Tensor] = None,):
        
        last_hidden_state = self.node_encoder(node_opcode,
                                    node_feat,
                                    edges_adjecency,
                                    node_attn_mask,
                                    node_config_feat,
                                    node_config_ids)
        
        output = self.head(last_hidden_state[:,:,0]).squeeze(-1)
        outputs = {'outputs': output, 'order': torch.argsort(output, dim=1)}
        if config_runtime is not None:
            loss = 0
            loss += self.loss_fn(output, config_runtime, config_idxs)
            outputs['loss'] = loss
        return outputs
"""
グラフのエンコーディングにBERTモデルのアーキテクチャを利用するカスタムニューラルネットワークモデルを定義しています。ノードとエッジの情報を処理して特徴量を抽出し、それを利用して特定のタスク（ここでは設定のランタイムを予測する）のための出力を生成します。コードは複数のクラスとメソッドで構成され、大量の情報を含んでいるため、以下に主要な部分を分解して説明します。

BertEncoder クラス 目的: BERTモデルのエンコーダ部分を定義しています。エンコーダは、入力テンソル（ノードとエッジの情報）を特徴ベクトルに変換する役割を果たします。 主要メソッド: forward は、エンコーディングの処理を行い、特徴ベクトル、注意の重み、隠れ状態などを返します。
BertLayer およびその関連クラス (BertIntermediate, BertOutput, BertAttention, BertSelfAttention, BertSelfOutput) 目的: BERTの内部層を定義しています。それぞれのクラスは、BERTのアテンションメカニズム、フィードフォワードネットワーク、正規化、ドロップアウトなどのコンポーネントを表現しています。 主要メソッド: 各クラスは forward メソッドを持っており、それぞれの部分の処理を行います。
NodeEncoder クラス 目的: グラフのノード情報をエンコードするためのクラスです。ノードのオペコードと特徴をエンコードして、それぞれのノードの特徴ベクトルを生成します。 主要メソッド: forward メソッドは、ノードのオペコードと特徴を受け取り、それをエンコードして特徴ベクトルを返します。
BertNodeEncoder クラス 目的: NodeEncoder と BertEncoder を組み合わせて、グラフのノードをエンコードするクラスです。 主要メソッド: forward メソッドは、ノードとエッジの情報を受け取り、それをエンコードして特徴ベクトルと注意の重みを返します。
transform_node_positional_embeddings 関数 この関数は、ノードの位置エンベッディングを変換する役割を果たしています。
パラメータ: embeddings_output: エンコーダからの出力エンベッディング。 node_config_ids: ノードの設定ID。 num_nodes: ノードの合計数。 動作: bs, numconfigs, , dim 変数は、エンベッディングの出力形状から得られます。bs はバッチサイズ、num_configs は設定の数、dim はエンベッディングの次元です。 idxs 変数は、ノード設定IDを用いて、各ノードの位置エンベッディングを取得するためのインデックスを生成します。 zeros は、変換後のエンベッディングを格納するためのゼロテンソルです。 最後に scatterreduce メソッドを用いて、embeddings_output からノードの位置エンベッディングを取得し、zeros テンソルに格納します。

NodeFeatEmbeddings クラス このクラスは、ノードとその特徴をエンコードし、位置エンベッディングを含む特徴ベクトルを生成します。
主要メソッド: forward メソッドは、ノードの設定特徴とIDを受け取り、それをエンコードして特徴ベクトルを生成します。transform_node_positional_embeddings 関数を使用して、位置エンベッディングを取得しています。

BertGraphEncoder クラス このクラスは、グラフのエンコーディング全体を担当します。ノードとエッジの情報、ノードの位置エンベッディングなどを処理し、グラフの特徴ベクトルを生成します。
主要メソッド: forward メソッドは、ノードとエッジの情報、ノードの設定情報などを受け取り、それをエンコードして特徴ベクトルを生成します。このクラスでは、ノードのエンコーディングと位置エンベッディングの両方を処理しています。

GraphEncoder クラス これは、グラフのエンコーディングと、そのエンコーディングを基にしたランタイムの予測を行うためのメインクラスです。
主要メソッド: forward メソッドは、グラフのノードとエッジの情報、ノードの設定情報、ランタイムなどを受け取り、エンコーディングとランタイムの予測を行います。また、必要に応じて、損失も計算します。 このクラスは、先に説明したBertGraphEncoderクラスを使用して、グラフをエンコードしています。さらに、MultiElementRankLoss損失関数を使用して、ランタイムの予測のための損失を計算します。
"""

class LightningWrapper(pl.LightningModule):
    def __init__(self, model:nn.Module):
        super().__init__()
        self.model = model
        self.topk = TileTopK()
        
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        outputs = self.model(**batch)
        return outputs['loss']

    def validation_step(self, batch, batch_idx):
        outputs = self.model(**batch)
        loss = outputs['loss']
        self.log("val_loss", loss, prog_bar=True)
        config_attn_mask = torch.ones_like(batch['config_runtime'], device=batch['config_runtime'].device)
        self.topk.update(outputs['outputs'], batch['config_runtime'], config_attn_mask)
        return loss
    
    def on_validation_end(self) -> None:
        topk = self.topk.compute()
        self.print(f"topk {topk:.3f}")
        self.topk.reset()
        return super().on_validation_end()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.model.loss(y_hat, y)
        self.log("test_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.trainer.model.parameters(), lr=1e-3)
        return optimizer
"""
LightningWrapper クラスの構造

init メソッド
このメソッドはクラスの初期化メソッドです。モデルとTop-K評価メトリックをインスタンス変数として保持します。

model: このインスタンス変数は、訓練、検証、テストを行うためのモデルを保持します。 topk: このインスタンス変数は、検証の際にモデルのパフォーマンスを評価するためのTop-Kメトリックを保持します。

forward メソッド
このメソッドは、モデルのフォワードパスを呼び出します。具体的には、入力xをモデルに渡して、出力を返します。

training_step メソッド
このメソッドは、各訓練ステップで呼び出され、バッチデータに対する訓練の損失を計算します。バッチデータはbatch引数として渡され、損失はモデルの出力から取得します。

validation_step メソッド
このメソッドは、各検証ステップで呼び出され、バッチデータに対する検証の損失を計算します。さらに、Top-Kメトリックもこのステップで計算され、更新されます。

loss: 検証の損失を計算します。 self.log: 検証の損失をログに記録します。 config_attn_mask: 設定の注目マスクを作成します。 self.topk.update: Top-Kメトリックを更新します。

on_validation_end メソッド
検証が終了したときに呼び出され、Top-Kメトリックの値を計算して表示します。その後、Top-Kメトリックをリセットします。

test_step メソッド
テストステップで呼び出され、バッチデータに対するテストの損失を計算します。

configure_optimizers メソッド
オプティマイザを設定するメソッドです。この例では、AdamWオプティマイザを使用しています。学習率は0.001に設定されています。

LightningWrapper クラスは、グラフエンコーダモデルをPyTorch Lightningフレームワークを使って効率的に訓練、検証、テストするためのユーティリティクラスです。訓練と検証のステップで損失を計算し、検証のステップでパフォーマンスメトリックも計算します。また、オプティマイザの設定やテストステップの実装も含まれています。
"""
#Training

config_kwargs = dict(hidden_size= 128,
    num_attention_heads= 4,
    num_hidden_layers= 2,
    intermediate_size= 64,
    gradient_checkpointing= True,
    margin= 0.1,
    number_permutations= 4,
    )
config = GraphConfig(**config_kwargs)
model = GraphEncoder(config)
model = LightningWrapper(model)
tile_df
"""
paths	split	configuration	extra	model_name	collection	ID
0	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	resnet_v1_50_official_batch_128_bf16_2bea628b7...	tile:xla	tile:xla:resnet_v1_50_official_batch_128_bf16_...
1	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	inception_v3_batch_128_train_40fa8f86f121f00a	tile:xla	tile:xla:inception_v3_batch_128_train_40fa8f86...
2	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	inception_v3_batch_128_train_-23e94c034a65a177	tile:xla	tile:xla:inception_v3_batch_128_train_-23e94c0...
3	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	inception_v3_batch_128_train_171f4371caf28639	tile:xla	tile:xla:inception_v3_batch_128_train_171f4371...
4	../input/predict-ai-model-runtime/npz_all/npz/...	valid	xla	tile	mlperf_bert_batch_24_2x2_-25e30862c042a2b8	tile:xla	tile:xla:mlperf_bert_batch_24_2x2_-25e30862c04...
...	...	...	...	...	...	...	...
7224	../input/predict-ai-model-runtime/npz_all/npz/...	train	xla	tile	shapemask.4x4.fp32_-308d824e29eea7d5	tile:xla	tile:xla:shapemask.4x4.fp32_-308d824e29eea7d5
7225	../input/predict-ai-model-runtime/npz_all/npz/...	train	xla	tile	mnasnet_b1_batch_128_274248815373b90a	tile:xla	tile:xla:mnasnet_b1_batch_128_274248815373b90a
7226	../input/predict-ai-model-runtime/npz_all/npz/...	train	xla	tile	shapemask.4x4.fp32_15c8ed14770f4c5e	tile:xla	tile:xla:shapemask.4x4.fp32_15c8ed14770f4c5e
7227	../input/predict-ai-model-runtime/npz_all/npz/...	train	xla	tile	inception_v2_batch_8_train_-2780d93f2933627	tile:xla	tile:xla:inception_v2_batch_8_train_-2780d93f2...
7228	../input/predict-ai-model-runtime/npz_all/npz/...	train	xla	tile	retinanet.4x4.fp32_-5ad42689cc8da2aa	tile:xla	tile:xla:retinanet.4x4.fp32_-5ad42689cc8da2aa
7229 rows × 7 columns
"""
train_df = tile_df.query("split == 'train'").reset_index(drop=True)
valid_df = tile_df.query("split == 'valid'").reset_index(drop=True)
train_dataset = TileDataset(train_df, num_configs=24)
valid_dataset = TileDataset(valid_df, num_configs=24)
train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=8, num_workers=2, shuffle=True, persistent_workers=True)
valid_dataloader = DataLoader(valid_dataset, collate_fn=collate_fn, batch_size=8, num_workers=2)
trainer_config = dict(
    max_epochs= 50,
    precision= 32,
    gradient_clip_val= 1.0,
    accumulate_grad_batches= 4,
    check_val_every_n_epoch= 10)
torch.set_float32_matmul_precision("medium")
trainer = pl.Trainer(**trainer_config,)
trainer.fit(model, train_dataloader, valid_dataloader)
"""
Epoch 49: 100%
714/714 [00:33<00:00, 21.61it/s, v_num=0, val_loss=0.0322]
topk 0.982
topk 0.983
topk 0.986
topk 0.987
topk 0.991
"""
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
split = 'test'
test_tile_df = tile_df.query("split == @split").reset_index(drop=True)
test_tile_ds = TileDataset(test_tile_df, num_configs=-1)
collate_fn = LayoutCollator(64, targets=split!="test")
test_dataloader = DataLoader(test_tile_ds, batch_size=1, shuffle=False, num_workers=0, collate_fn=collate_fn)
model.to(device)
model = model.eval()
def chunk_batch(batch, start_idx, end_idx):
    output = {k:batch[k] for k in ['node_opcode', 'node_feat', 'edges_adjecency', 'node_attn_mask', 'node_config_ids']}
    output['node_config_feat'] = batch['node_config_feat'][:, start_idx: end_idx]
    return output
    
pred_order = []
for batch in tqdm(test_dataloader):
    batch.pop('config_idxs')
    batch = {k: v.to(device) for k, v in batch.items()}
    num_configs = batch['node_config_feat'].shape[1]
    # Chunk the configs to avoid OOM errors
    configs_cut_points = list(range(0,num_configs, 100)) + [num_configs]
    chunk_order = []
    for start, end in zip(configs_cut_points, configs_cut_points[1:]):
        chunked_batch = chunk_batch(batch, start, end)
        with torch.no_grad():
            output = model.model(**chunked_batch)
        chunk_order.extend(output['outputs'].cpu().numpy())
    pred_order.append(np.argsort(np.concatenate(chunk_order))[:5])
"""
100%|██████████| 844/844 [01:18<00:00, 10.81it/s]
"""
idxs_string = [";".join(map(str,elem)) for elem in pred_order]
test_tile_df['TopConfigs'] = idxs_string
test_tile_df = test_tile_df[['ID', 'TopConfigs']]
test_tile_df.head()
"""
ID	TopConfigs
0	tile:xla:04ae9238c653f8ae08f60f2c03615f0b	273;299;479;661;385
1	tile:xla:85d157d3b1848c6b6fff0c633876e2e6	6792;8019;7513;2902;3531
2	tile:xla:862900d42397d03be2762e1bf7518bea	206;161;1409;287;1344
3	tile:xla:0afa527a7022415fda1dd69d11e908a4	158;210;212;69;20
4	tile:xla:2d09e3ab92e184c561abaf8d9efe7b87	170;147;24;89;6
"""
submission_df = pd.read_csv('../input/predict-ai-model-runtime/sample_submission.csv')
submission_df = submission_df.query(f"ID not in {test_tile_df.ID.tolist()}")
submission_df = pd.concat([test_tile_df, submission_df])
submission_df.to_csv('submission.csv', index=False)
submission_df
"""
ID	TopConfigs
0	tile:xla:04ae9238c653f8ae08f60f2c03615f0b	273;299;479;661;385
1	tile:xla:85d157d3b1848c6b6fff0c633876e2e6	6792;8019;7513;2902;3531
2	tile:xla:862900d42397d03be2762e1bf7518bea	206;161;1409;287;1344
3	tile:xla:0afa527a7022415fda1dd69d11e908a4	158;210;212;69;20
4	tile:xla:2d09e3ab92e184c561abaf8d9efe7b87	170;147;24;89;6
...	...	...
889	layout:nlp:random:60880ed76de53f4d7a1b960b24f2...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
890	layout:nlp:random:23559853d9702baaaacbb0c83fd3...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
891	layout:nlp:random:f6c146fc5cf10be4f3accbaca989...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
892	layout:nlp:random:32531d07a084b319dce484f53a4c...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
893	layout:nlp:random:3a0c5517a87df8d82fd637b83298...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
894 rows × 2 columns
"""

!pip install /kaggle/input/fast-slow-4-dataset-train/torch_geometric-2.3.1-py3-none-any.whl
!pip install /kaggle/input/fast-slow-4-dataset-train/torch_scatter-2.1.1-cp310-cp310-linux_x86_64.whl
"""
Processing /kaggle/input/fast-slow-4-dataset-train/torch_geometric-2.3.1-py3-none-any.whl
Requirement already satisfied: tqdm in /opt/conda/lib/python3.10/site-packages (from torch-geometric==2.3.1) (4.66.1)
Requirement already satisfied: numpy in /opt/conda/lib/python3.10/site-packages (from torch-geometric==2.3.1) (1.23.5)
Requirement already satisfied: scipy in /opt/conda/lib/python3.10/site-packages (from torch-geometric==2.3.1) (1.11.2)
Requirement already satisfied: jinja2 in /opt/conda/lib/python3.10/site-packages (from torch-geometric==2.3.1) (3.1.2)
Requirement already satisfied: requests in /opt/conda/lib/python3.10/site-packages (from torch-geometric==2.3.1) (2.31.0)
Requirement already satisfied: pyparsing in /opt/conda/lib/python3.10/site-packages (from torch-geometric==2.3.1) (3.0.9)
Requirement already satisfied: scikit-learn in /opt/conda/lib/python3.10/site-packages (from torch-geometric==2.3.1) (1.2.2)
Requirement already satisfied: psutil>=5.8.0 in /opt/conda/lib/python3.10/site-packages (from torch-geometric==2.3.1) (5.9.3)
Requirement already satisfied: MarkupSafe>=2.0 in /opt/conda/lib/python3.10/site-packages (from jinja2->torch-geometric==2.3.1) (2.1.3)
Requirement already satisfied: charset-normalizer<4,>=2 in /opt/conda/lib/python3.10/site-packages (from requests->torch-geometric==2.3.1) (3.1.0)
Requirement already satisfied: idna<4,>=2.5 in /opt/conda/lib/python3.10/site-packages (from requests->torch-geometric==2.3.1) (3.4)
Requirement already satisfied: urllib3<3,>=1.21.1 in /opt/conda/lib/python3.10/site-packages (from requests->torch-geometric==2.3.1) (1.26.15)
Requirement already satisfied: certifi>=2017.4.17 in /opt/conda/lib/python3.10/site-packages (from requests->torch-geometric==2.3.1) (2023.7.22)
Requirement already satisfied: joblib>=1.1.1 in /opt/conda/lib/python3.10/site-packages (from scikit-learn->torch-geometric==2.3.1) (1.3.2)
Requirement already satisfied: threadpoolctl>=2.0.0 in /opt/conda/lib/python3.10/site-packages (from scikit-learn->torch-geometric==2.3.1) (3.1.0)
Installing collected packages: torch-geometric
Successfully installed torch-geometric-2.3.1
Processing /kaggle/input/fast-slow-4-dataset-train/torch_scatter-2.1.1-cp310-cp310-linux_x86_64.whl
Installing collected packages: torch-scatter
Successfully installed torch-scatter-2.1.1
!pip install timm
Requirement already satisfied: timm in /opt/conda/lib/python3.10/site-packages (0.9.7)
Requirement already satisfied: torch>=1.7 in /opt/conda/lib/python3.10/site-packages (from timm) (2.0.0)
Requirement already satisfied: torchvision in /opt/conda/lib/python3.10/site-packages (from timm) (0.15.1)
Requirement already satisfied: pyyaml in /opt/conda/lib/python3.10/site-packages (from timm) (6.0)
Requirement already satisfied: huggingface-hub in /opt/conda/lib/python3.10/site-packages (from timm) (0.16.4)
Requirement already satisfied: safetensors in /opt/conda/lib/python3.10/site-packages (from timm) (0.3.3)
Requirement already satisfied: filelock in /opt/conda/lib/python3.10/site-packages (from torch>=1.7->timm) (3.12.2)
Requirement already satisfied: typing-extensions in /opt/conda/lib/python3.10/site-packages (from torch>=1.7->timm) (4.6.3)
Requirement already satisfied: sympy in /opt/conda/lib/python3.10/site-packages (from torch>=1.7->timm) (1.12)
Requirement already satisfied: networkx in /opt/conda/lib/python3.10/site-packages (from torch>=1.7->timm) (3.1)
Requirement already satisfied: jinja2 in /opt/conda/lib/python3.10/site-packages (from torch>=1.7->timm) (3.1.2)
Requirement already satisfied: fsspec in /opt/conda/lib/python3.10/site-packages (from huggingface-hub->timm) (2023.9.0)
Requirement already satisfied: requests in /opt/conda/lib/python3.10/site-packages (from huggingface-hub->timm) (2.31.0)
Requirement already satisfied: tqdm>=4.42.1 in /opt/conda/lib/python3.10/site-packages (from huggingface-hub->timm) (4.66.1)
Requirement already satisfied: packaging>=20.9 in /opt/conda/lib/python3.10/site-packages (from huggingface-hub->timm) (21.3)
Requirement already satisfied: numpy in /opt/conda/lib/python3.10/site-packages (from torchvision->timm) (1.23.5)
Requirement already satisfied: pillow!=8.3.*,>=5.3.0 in /opt/conda/lib/python3.10/site-packages (from torchvision->timm) (9.5.0)
Requirement already satisfied: pyparsing!=3.0.5,>=2.0.2 in /opt/conda/lib/python3.10/site-packages (from packaging>=20.9->huggingface-hub->timm) (3.0.9)
Requirement already satisfied: MarkupSafe>=2.0 in /opt/conda/lib/python3.10/site-packages (from jinja2->torch>=1.7->timm) (2.1.3)
Requirement already satisfied: charset-normalizer<4,>=2 in /opt/conda/lib/python3.10/site-packages (from requests->huggingface-hub->timm) (3.1.0)
Requirement already satisfied: idna<4,>=2.5 in /opt/conda/lib/python3.10/site-packages (from requests->huggingface-hub->timm) (3.4)
Requirement already satisfied: urllib3<3,>=1.21.1 in /opt/conda/lib/python3.10/site-packages (from requests->huggingface-hub->timm) (1.26.15)
Requirement already satisfied: certifi>=2017.4.17 in /opt/conda/lib/python3.10/site-packages (from requests->huggingface-hub->timm) (2023.7.22)
Requirement already satisfied: mpmath>=0.19 in /opt/conda/lib/python3.10/site-packages (from sympy->torch>=1.7->timm) (1.3.0)
"""
import timm
from timm.scheduler import  CosineLRScheduler
import numpy as np
import pandas as pd
import os
from tqdm import tqdm 

import sklearn,sklearn.model_selection
import torch
from torch import nn
from torch import Tensor
from torch_geometric.nn import GCNConv,SAGEConv
from torch_geometric.datasets import Planetoid
from torch.utils.data import DataLoader, Dataset
#from timm.scheduler import CosineLRScheduler
import matplotlib.pyplot as plt
device = 'cpu'
def load_df(directory):
    splits = ["test"]
    dfs = dict()
    
    for split in splits:
        path = os.path.join(directory, split)
        files = os.listdir(path)
        list_df = []
        
        for file in files:
            d = dict(np.load(os.path.join(path,file)))
            d['file'] = file
            list_df.append(d)
        dfs[split] = pd.DataFrame.from_dict(list_df)
    return dfs
layout_xla_random = load_df("/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/random/")
layout_xla_default = load_df("/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/default/")
layout_nlp_default = load_df("/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default/")
layout_nlp_random = load_df("/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/")
class TileDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        config_feat = torch.tensor(row['node_config_feat'].astype(np.float32))
        node_feat = torch.tensor(row['node_feat'].astype(np.float32))
        node_opcode = torch.tensor(row['node_opcode'].astype(np.int64))
        edge_index = torch.tensor(np.swapaxes(row['edge_index'],0,1).astype(np.int64))
        target = (row['config_runtime']).astype(np.float32)
        # minmax scale the target, we only care about order
        target = (target-min(target))/(max(target) -min(target))
        target = torch.tensor(target)
        return config_feat,node_feat,node_opcode,edge_index,target
    
class SimpleModel(torch.nn.Module):
    def __init__(self, hidden_channels, graph_feats, hidden_dim):
        super().__init__()
        op_embedding_dim = 4 # I choose 4-dimensional embedding
        self.embedding = torch.nn.Embedding(120, #120 different op-codes
                                            op_embedding_dim,
                                           )
        assert len(hidden_channels)>0
        in_channels = op_embedding_dim+140
        self.convs = torch.nn.ModuleList()
        last_dim = hidden_channels[0]
        self.convs.append(GCNConv(in_channels, hidden_channels[0]))
        for i in range(len(hidden_channels)-1):
            self.convs.append(GCNConv(hidden_channels[i], hidden_channels[i+1]))
            last_dim = hidden_channels[i+1]
        self.convs.append(GCNConv(last_dim, graph_feats))
        
        self.dense = torch.nn.Sequential(nn.Linear(82, 64),
                                         nn.ReLU(),
                                         nn.Linear(64, 64),
                                         nn.ReLU(),
                                         nn.Linear(64, 1),
                                        )
    
    def forward(self, x_cfg: Tensor,x_feat: Tensor, x_op: Tensor, edge_index: Tensor) -> Tensor:
        
        #get graph features
        x_cfg = x_cfg.mean(dim=1)
        #print(x_cfg.shape)
        x = torch.concat([x_feat,self.embedding(x_op)],dim = 1)
        #pass though conv layers
        for conv in self.convs:
            x = conv(x, edge_index).relu()
        # get 1d graph embedding using average pooling
        x_graph = torch.mean(x,0)
        
        
        #put graph data into config data
        x = torch.concat([x_cfg,x_graph.repeat((len(x_cfg),1))],axis=1) #torch.Size([10528, 225])
        #put into dense nn
        #print(x.shape)
        x = torch.flatten(self.dense(x))
        return x

model = SimpleModel(hidden_channels = [16,32,16,48],graph_feats = 64,hidden_dim=64).to(device)
class SimpleModel2(torch.nn.Module):
    def __init__(self, hidden_channels, graph_feats, hidden_dim):
        super().__init__()
        op_embedding_dim = 4 # I choose 4-dimensional embedding
        self.embedding = torch.nn.Embedding(120, #120 different op-codes
                                            op_embedding_dim,
                                           )
        assert len(hidden_channels)>0
        in_channels = op_embedding_dim+140
        self.convs = torch.nn.ModuleList()
        last_dim = hidden_channels[0]
        self.convs.append(SAGEConv(in_channels, hidden_channels[0]))
        for i in range(len(hidden_channels)-1):
            self.convs.append(SAGEConv(hidden_channels[i], hidden_channels[i+1]))
            last_dim = hidden_channels[i+1]
        self.convs.append(SAGEConv(last_dim, graph_feats))
        
        self.dense = torch.nn.Sequential(nn.Linear(82, 64),
                                         nn.ReLU(),
                                         nn.Linear(64, 64),
                                         nn.ReLU(),
                                         nn.Linear(64, 1),
                                        )
    
    def forward(self, x_cfg: Tensor,x_feat: Tensor, x_op: Tensor, edge_index: Tensor) -> Tensor:
        
        #get graph features
        x_cfg = x_cfg.mean(dim=1)
        #print(x_cfg.shape)
        x = torch.concat([x_feat,self.embedding(x_op)],dim = 1)
        #pass though conv layers
        for conv in self.convs:
            x = conv(x, edge_index).relu()
        # get 1d graph embedding using average pooling
        x_graph = torch.mean(x,0)
        
        
        #put graph data into config data
        x = torch.concat([x_cfg,x_graph.repeat((len(x_cfg),1))],axis=1) #torch.Size([10528, 225])
        #put into dense nn
        #print(x.shape)
        x = torch.flatten(self.dense(x))
        return x

model2 = SimpleModel2(hidden_channels = [16,32,16,48],graph_feats = 64,hidden_dim=64).to(device)
dataset = TileDataset(layout_xla_default["test"])
tile_xla_predictions = [[] for i in range(len(dataset))]
for fold in range(5):
    model.load_state_dict(torch.load(f'/kaggle/input/fast-slow-sep/xla_defalut/layout_xla_default_best_model_{fold}.pth',map_location=torch.device('cpu') ))
    model.eval()
    pbar = tqdm(range(len(dataset)))
    for i in pbar:
        cfg_ft,nd_ft,nd_op,ind,target = dataset[i]
        cfg_ft,nd_ft,nd_op,ind,target = cfg_ft.to(device),nd_ft.to(device),nd_op.to(device),ind.to(device),target.to(device)

        out = model(cfg_ft,nd_ft,nd_op,ind)
        tile_xla_predictions[i].append(out.cpu().detach().numpy())
tile_xla_predictions = [np.argsort(np.mean(pred,axis=0))[:-1] for pred in tile_xla_predictions]
tile_xla_predictions[0]
#sub = submission_df
sub = pd.read_csv('/kaggle/input/predict-ai-model-runtime/sample_submission.csv')
for i,filename in enumerate(layout_xla_random["test"]['file'].values):
    id = 'layout:xla:default:' +filename[:-4]
    print(id)
    sub.loc[sub.ID == id,'TopConfigs'] = ';'.join(tile_xla_predictions[i].astype(str))
sub.to_csv('submission.csv',index=False)
sub
"""
  0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_23/1750813363.py:16: RuntimeWarning: invalid value encountered in divide
  target = (target-min(target))/(max(target) -min(target))
100%|██████████| 8/8 [00:01<00:00,  5.14it/s]
100%|██████████| 8/8 [00:01<00:00,  5.07it/s]
100%|██████████| 8/8 [00:01<00:00,  5.20it/s]
100%|██████████| 8/8 [00:01<00:00,  5.45it/s]
100%|██████████| 8/8 [00:01<00:00,  5.45it/s]
layout:xla:default:cd708819d3f5103afd6460b15e74eaf3
layout:xla:default:05ae41e26dd3c4c06390371a0423233c
layout:xla:default:e8a3a1401b5e79f66d7037e424f3b6df
layout:xla:default:fbaa8bb6a1aed9988281085c91065c05
layout:xla:default:937ee0eb0d5d6151b7b8252933b5c1c9
layout:xla:default:3e7156ac468dfb75cf5c9615e1e5887d
layout:xla:default:5335ed13823b0a518ee3c79ba4425f34
layout:xla:default:db59a991b7c607634f13570d52ce885f
ID	TopConfigs
0	tile:xla:d6f5f54247bd1e58a10b9e7062c636ab	0;1;2;3;4
1	tile:xla:e3a655daa38e34ec240df959b650ac16	0;1;2;3;4
2	tile:xla:f8c2c1a1098b2a361c26df668b286c87	0;1;2;3;4
3	tile:xla:4dd1716853ed46ee4e7d09ede1732de8	0;1;2;3;4
4	tile:xla:d0a69155b6340748c36724e4bfc34be3	0;1;2;3;4
...	...	...
889	layout:nlp:random:60880ed76de53f4d7a1b960b24f2...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
890	layout:nlp:random:23559853d9702baaaacbb0c83fd3...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
891	layout:nlp:random:f6c146fc5cf10be4f3accbaca989...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
892	layout:nlp:random:32531d07a084b319dce484f53a4c...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
893	layout:nlp:random:3a0c5517a87df8d82fd637b83298...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
894 rows × 2 columns
"""
dataset = TileDataset(layout_xla_random["test"])
tile_xla_predictions = [[] for i in range(len(dataset))]
for fold in range(5):
    model.load_state_dict(torch.load(f'/kaggle/input/fast-slow-sep/xla_random/layout_xla_default_best_model_{fold}.pth',map_location=torch.device('cpu') ))
    model.eval()
    pbar = tqdm(range(len(dataset)))
    for i in pbar:
        cfg_ft,nd_ft,nd_op,ind,target = dataset[i]
        cfg_ft,nd_ft,nd_op,ind,target = cfg_ft.to(device),nd_ft.to(device),nd_op.to(device),ind.to(device),target.to(device)

        out = model(cfg_ft,nd_ft,nd_op,ind)
        tile_xla_predictions[i].append(out.cpu().detach().numpy())
tile_xla_predictions = [np.argsort(np.mean(pred,axis=0))[:-1] for pred in tile_xla_predictions]
tile_xla_predictions[0]

#sub = pd.read_csv('/kaggle/input/predict-ai-model-runtime/sample_submission.csv')
for i,filename in enumerate(layout_xla_random["test"]['file'].values):
    id = 'layout:xla:random:' +filename[:-4]
    print(id)
    sub.loc[sub.ID == id,'TopConfigs'] = ';'.join(tile_xla_predictions[i].astype(str))
sub.to_csv('submission.csv',index=False)
sub
"""
  0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_23/1750813363.py:16: RuntimeWarning: invalid value encountered in divide
  target = (target-min(target))/(max(target) -min(target))
100%|██████████| 8/8 [00:01<00:00,  5.35it/s]
100%|██████████| 8/8 [00:01<00:00,  5.63it/s]
100%|██████████| 8/8 [00:02<00:00,  3.33it/s]
100%|██████████| 8/8 [00:01<00:00,  5.74it/s]
100%|██████████| 8/8 [00:01<00:00,  5.69it/s]
layout:xla:random:cd708819d3f5103afd6460b15e74eaf3
layout:xla:random:05ae41e26dd3c4c06390371a0423233c
layout:xla:random:e8a3a1401b5e79f66d7037e424f3b6df
layout:xla:random:fbaa8bb6a1aed9988281085c91065c05
layout:xla:random:937ee0eb0d5d6151b7b8252933b5c1c9
layout:xla:random:3e7156ac468dfb75cf5c9615e1e5887d
layout:xla:random:5335ed13823b0a518ee3c79ba4425f34
layout:xla:random:db59a991b7c607634f13570d52ce885f
ID	TopConfigs
0	tile:xla:d6f5f54247bd1e58a10b9e7062c636ab	0;1;2;3;4
1	tile:xla:e3a655daa38e34ec240df959b650ac16	0;1;2;3;4
2	tile:xla:f8c2c1a1098b2a361c26df668b286c87	0;1;2;3;4
3	tile:xla:4dd1716853ed46ee4e7d09ede1732de8	0;1;2;3;4
4	tile:xla:d0a69155b6340748c36724e4bfc34be3	0;1;2;3;4
...	...	...
889	layout:nlp:random:60880ed76de53f4d7a1b960b24f2...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
890	layout:nlp:random:23559853d9702baaaacbb0c83fd3...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
891	layout:nlp:random:f6c146fc5cf10be4f3accbaca989...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
892	layout:nlp:random:32531d07a084b319dce484f53a4c...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
893	layout:nlp:random:3a0c5517a87df8d82fd637b83298...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
894 rows × 2 columns
"""
dataset = TileDataset(layout_nlp_default["test"])
tile_xla_predictions = [[] for i in range(len(dataset))]
for fold in range(5):
    model.load_state_dict(torch.load(f'/kaggle/input/fast-slow-nlp-v3/nlp_default/layout_xla_default_best_model_{fold}.pth',map_location=torch.device('cpu') ))
    model.eval()
    pbar = tqdm(range(len(dataset)))
    for i in pbar:
        cfg_ft,nd_ft,nd_op,ind,target = dataset[i]
        cfg_ft,nd_ft,nd_op,ind,target = cfg_ft.to(device),nd_ft.to(device),nd_op.to(device),ind.to(device),target.to(device)
        out = model(cfg_ft,nd_ft,nd_op,ind) 
        tile_xla_predictions[i].append(out.cpu().detach().numpy())
tile_xla_predictions = [np.argsort(np.mean(pred,axis=0))[:-1] for pred in tile_xla_predictions]
tile_xla_predictions[0]

#sub = pd.read_csv('/kaggle/input/predict-ai-model-runtime/sample_submission.csv')
for i,filename in enumerate(layout_nlp_default["test"]['file'].values):
    id = 'layout:nlp:default:' +filename[:-4]
    print(id)
    sub.loc[sub.ID == id,'TopConfigs'] = ';'.join(tile_xla_predictions[i].astype(str))
sub.to_csv('submission.csv',index=False)
sub
"""
  0%|          | 0/17 [00:00<?, ?it/s]/tmp/ipykernel_23/1750813363.py:16: RuntimeWarning: invalid value encountered in divide
  target = (target-min(target))/(max(target) -min(target))
100%|██████████| 17/17 [00:00<00:00, 42.06it/s]
100%|██████████| 17/17 [00:00<00:00, 40.82it/s]
100%|██████████| 17/17 [00:00<00:00, 33.71it/s]
100%|██████████| 17/17 [00:00<00:00, 36.62it/s]
100%|██████████| 17/17 [00:00<00:00, 42.31it/s]
layout:nlp:default:b2fdde3b72980907578648774101543e
layout:nlp:default:29886a50d55cfe77a9497bc906c76ce9
layout:nlp:default:7105451001e119f65b66570d170b94a8
layout:nlp:default:171b0513d8874a427ccfa46d136fbadc
layout:nlp:default:60880ed76de53f4d7a1b960b24f20f7d
layout:nlp:default:58cc2e418c3a8a19b871e15964b534ad
layout:nlp:default:f6c146fc5cf10be4f3accbaca9897311
layout:nlp:default:38524e2ff135ded55b5286407e7af6b7
layout:nlp:default:3a0c5517a87df8d82fd637b83298a3ba
layout:nlp:default:6c1101f6231f4d1722c3b9f6d1e25026
layout:nlp:default:016ac66a44a906a695afd2228509046a
layout:nlp:default:492c7a94d559aa4a88769142d2a68362
layout:nlp:default:d15316c12eefdef1ba549eb433797f77
layout:nlp:default:7f6284ebe027b1e9a3850fc703858a59
layout:nlp:default:32531d07a084b319dce484f53a4cf3fc
layout:nlp:default:23559853d9702baaaacbb0c83fd32266
layout:nlp:default:71b79ca6db513e7979c3702c595150c2
ID	TopConfigs
0	tile:xla:d6f5f54247bd1e58a10b9e7062c636ab	0;1;2;3;4
1	tile:xla:e3a655daa38e34ec240df959b650ac16	0;1;2;3;4
2	tile:xla:f8c2c1a1098b2a361c26df668b286c87	0;1;2;3;4
3	tile:xla:4dd1716853ed46ee4e7d09ede1732de8	0;1;2;3;4
4	tile:xla:d0a69155b6340748c36724e4bfc34be3	0;1;2;3;4
...	...	...
889	layout:nlp:random:60880ed76de53f4d7a1b960b24f2...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
890	layout:nlp:random:23559853d9702baaaacbb0c83fd3...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
891	layout:nlp:random:f6c146fc5cf10be4f3accbaca989...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
892	layout:nlp:random:32531d07a084b319dce484f53a4c...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
893	layout:nlp:random:3a0c5517a87df8d82fd637b83298...	0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
894 rows × 2 columns

43のコードをオリジナルでエラーが出たので改変した。
"""
dataset = TileDataset(layout_nlp_random["test"])
tile_xla_predictions = [[] for i in range(len(dataset))]
for fold in range(2):
    model.load_state_dict(torch.load(f'/kaggle/input/fast-slow-sep/nlp_random/layout_xla_default_best_model_{fold}.pth',map_location=torch.device('cpu') ))
    model.eval()
    
    pbar = tqdm(range(len(dataset)))
    for i in pbar:
        cfg_ft,nd_ft,nd_op,ind,target = dataset[i]
        cfg_ft,nd_ft,nd_op,ind,target = cfg_ft.to(device),nd_ft.to(device),nd_op.to(device),ind.to(device),target.to(device)

        out = model(cfg_ft,nd_ft,nd_op,ind) 
        tile_xla_predictions[i].append(out.cpu().detach().numpy())
tile_xla_predictions = [np.argsort(np.mean(pred,axis=0))[:-1] for pred in tile_xla_predictions]
tile_xla_predictions[0]

#sub = pd.read_csv('/kaggle/input/predict-ai-model-runtime/sample_submission.csv')
for i,filename in enumerate(layout_nlp_random["test"]['file'].values):
    id = 'layout:nlp:random:' +filename[:-4]
    print(id)
    sub.loc[sub.ID == id,'TopConfigs'] = ';'.join(tile_xla_predictions[i].astype(str))
sub.to_csv('submission.csv',index=False)
sub
"""
  0%|          | 0/17 [00:00<?, ?it/s]/tmp/ipykernel_23/1750813363.py:16: RuntimeWarning: invalid value encountered in divide
  target = (target-min(target))/(max(target) -min(target))
100%|██████████| 17/17 [00:00<00:00, 38.68it/s]
100%|██████████| 17/17 [00:00<00:00, 39.14it/s]
layout:nlp:random:b2fdde3b72980907578648774101543e
layout:nlp:random:29886a50d55cfe77a9497bc906c76ce9
layout:nlp:random:7105451001e119f65b66570d170b94a8
layout:nlp:random:171b0513d8874a427ccfa46d136fbadc
layout:nlp:random:60880ed76de53f4d7a1b960b24f20f7d
layout:nlp:random:58cc2e418c3a8a19b871e15964b534ad
layout:nlp:random:f6c146fc5cf10be4f3accbaca9897311
layout:nlp:random:38524e2ff135ded55b5286407e7af6b7
layout:nlp:random:3a0c5517a87df8d82fd637b83298a3ba
layout:nlp:random:6c1101f6231f4d1722c3b9f6d1e25026
layout:nlp:random:016ac66a44a906a695afd2228509046a
layout:nlp:random:492c7a94d559aa4a88769142d2a68362
layout:nlp:random:d15316c12eefdef1ba549eb433797f77
layout:nlp:random:7f6284ebe027b1e9a3850fc703858a59
layout:nlp:random:32531d07a084b319dce484f53a4cf3fc
layout:nlp:random:23559853d9702baaaacbb0c83fd32266
layout:nlp:random:71b79ca6db513e7979c3702c595150c2
ID	TopConfigs
0	tile:xla:d6f5f54247bd1e58a10b9e7062c636ab	0;1;2;3;4
1	tile:xla:e3a655daa38e34ec240df959b650ac16	0;1;2;3;4
2	tile:xla:f8c2c1a1098b2a361c26df668b286c87	0;1;2;3;4
3	tile:xla:4dd1716853ed46ee4e7d09ede1732de8	0;1;2;3;4
4	tile:xla:d0a69155b6340748c36724e4bfc34be3	0;1;2;3;4
...	...	...
889	layout:nlp:random:60880ed76de53f4d7a1b960b24f2...	17;604;850;9;150;326;639;288;23;286;627;923;57...
890	layout:nlp:random:23559853d9702baaaacbb0c83fd3...	62;976;644;52;218;38;30;275;916;318;618;74;893...
891	layout:nlp:random:f6c146fc5cf10be4f3accbaca989...	208;615;715;367;967;594;408;363;649;282;767;55...
892	layout:nlp:random:32531d07a084b319dce484f53a4c...	212;76;849;778;847;4;396;482;787;605;934;273;7...
893	layout:nlp:random:3a0c5517a87df8d82fd637b83298...	869;518;130;854;967;998;876;240;582;187;573;25...
894 rows × 2 columns

"""
 
 